# Individual Project
# Formal modelling and statistical analysis of TOR

### Author: Leonidas Reppas
### Supervisor: Gethin Norman

#### Libraries:

In [2]:
import random

#### Relay statistics from the TOR network for 2019 (gathered from Tor metrics):
* Roughly 6000 relays
* 2500 guard-flagged
* 1000 exit-flagged
* Users: 2,000,000

In [4]:
# Network setup / Relay creation

# Have different global variables for differently flagged nodes
total_nodes = 0
guard_nodes = 0
exit_nodes = 0
guard_exit_nodes = 0
middle_nodes = 0

class Relays():
    '''
    This class deals with the creation of the relay types
    '''
    
    def create_guard_nodes(self, number):
        global guard_nodes, total_nodes
        guard_nodes += number
        
    def create_exit_nodes(self, number):
        global exit_nodes, total_nodes
        exit_nodes += number
    
    def create_guard_exit_nodes(self, number):
        global guard_exit_nodes, total_nodes
        guard_exit_nodes += number
    
    def create_middle_nodes(self, number):
        global middle_nodes
        middle_nodes += number
        
    def calculate_total_nodes(self):
        global guard_nodes, exit_nodes, guard_exit_nodes, middle_nodes, total_nodes
        total_nodes = (guard_nodes + exit_nodes + middle_nodes) - guard_exit_nodes
        
    def print_nodes(self):
        global guard_nodes, exit_nodes, guard_exit_nodes, middle_nodes, total_nodes
        
        print("""
        Number of guard nodes: {}
        Number of exit nodes: {}
        Number of guard and exit nodes: {}
        Number of middle nodes: {}\n
        Number of total nodes in the network: {}
        """. format(guard_nodes, exit_nodes, guard_exit_nodes, middle_nodes, total_nodes))
            
# Testing
relay = Relays()
relay.create_guard_nodes(2500)
relay.create_exit_nodes(1000)
relay.create_guard_exit_nodes(250)
relay.create_middle_nodes(2750)
relay.calculate_total_nodes()
relay.print_nodes()


        Number of guard nodes: 2500
        Number of exit nodes: 1000
        Number of guard and exit nodes: 250
        Number of middle nodes: 2750

        Number of total nodes in the network: 6000
        


#### Path selection algorithm implemented:
 * Node selection based on weights, calculated by node bandwidth.
This was an improvement to the uniformly random path selection. Now a node with 10x higher bandwidth than another node,
has 10x as many circuits and probabilistically 10x as much of the traffic. 

#### Bandwidth weight computation according to the network scarcity conditions:
* G: total bandwidth for guard-flagged nodes = 230Gbit/s
* M: total bandwidth for non-flagged nodes = 50Gbit/s
* E: total bandwidth for exit-flagged nodes = 275Gbit/s
* D: total bandwidth for guard&exit-flagged nodes = 125Gbit/s
* T = G + M + E + D = 680Gbit/s

Network scarcity condition: E >= T/3 && G >= T/3 (275>227 && 230>227)
So weight solution is(weight_scale=10,000):
* Wgd = weight_scale/3
* Wed = weight_scale/3
* Wmd = weight_scale/3
* Wee = (weight_scale*(E+G+M))/(3*E)
* Wme = weight_scale - Wee
* Wmg = (weight_scale*(2*G-E-M))/(3*G)
* Wgg = weight_scale - Wmg
* Wgm = Wgg
* Wmm = Wbm
* Weg = Wed
* Wem = Wee

In [1]:
# Defining bandwidth values as global variables (in Gbit/s)
G = 230
M = 50
E = 275
D = 125
T = G + M + E + D

In [9]:
# Path Selection Algorithm 
class TorPath():
    '''
    This class deals with the bandwidth weight path 
    selection algorithm for the different user models defined
    '''
    weight_scale = 10000
    w_gd, w_ed, w_md = 0
    w_ee, w_me = 0
    w_mg, w_gg, w_eg = 0
    w_gm, w_mm, w_em = 0
    
    def calculate_bandwidth_weights(self):
        w_gd = weight_scale / 3
        w_ed = weight_scale / 3
        w_md = weight_scale / 3
        w_ee = (weight_scale * (E + G + M)) / (3 * E)
        w_me = weight_scale - w_ee
        w_mg = (weight_scale * (2 * G - E - M)) / (3 * G)
        w_gg = weight_scale - w_mg
        w_gm = w_gg
        w_mm = w_bm //TODO
        w_eg = w_ed
        w_em = w_ee
    
    
    def create_streams_typical(self, number):
        # select an exit node
        # select a middle node
        # select three guard nodes
        
    def create_streams_irc(self, number):
        
    def create_streams_bittorrent(self, number):
                    
            
            
# Testing - 1 week simulation per user model
torpath = TorPath()
torpath.create_streams_typical(1)

Path #1: entry: high, middle: high, exit: med
Path #2: entry: high, middle: med, exit: high
Path #3: entry: low, middle: med, exit: med
Path #4: entry: med, middle: med, exit: med
Path #5: entry: high, middle: high, exit: med
Path #6: entry: high, middle: high, exit: high
Path #7: entry: high, middle: low, exit: high
Path #8: entry: low, middle: high, exit: med
Path #9: entry: low, middle: high, exit: high
Path #10: entry: high, middle: high, exit: high
